#### Training Evaluation

In [1]:
# train_subgraph_gnn_fixed.py
import os
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict

import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import SAGEConv, global_mean_pool

RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 32
EPOCHS = 80
LR = 1e-3
HID_DIM = 64

# -----------------------
# Load node features
# -----------------------
nodes_df = pd.read_csv("GNNDatasets/node.csv")
nodes_df["uid"] = nodes_df["circuit_name"].astype(str) + "::" + nodes_df["node"].astype(str)

# one-hot gate_type
if "gate_type" in nodes_df.columns:
    gate_ohe = pd.get_dummies(nodes_df["gate_type"], prefix="gt")
    nodes_feat_df = pd.concat([nodes_df.drop(columns=["gate_type"]), gate_ohe], axis=1)
else:
    nodes_feat_df = nodes_df.copy()

meta_cols = {"uid","node","circuit_name","label","label_node","label_graph","label_subgraph","folder"}
num_cols = [c for c in nodes_feat_df.columns if c not in meta_cols and pd.api.types.is_numeric_dtype(nodes_feat_df[c])]

uid_to_feat = {}
for _, r in nodes_feat_df.iterrows():
    uid_to_feat[r["uid"]] = r[num_cols].astype(float).values

scaler = StandardScaler().fit(np.stack(list(uid_to_feat.values())))
for k in list(uid_to_feat.keys()):
    uid_to_feat[k] = scaler.transform(uid_to_feat[k].reshape(1,-1)).reshape(-1)

feat_dim = len(uid_to_feat[list(uid_to_feat.keys())[0]])

# -----------------------
# Merge edge CSVs
# -----------------------
edge_files = [
    "GNNDatasets/node_edges.csv",
    "GNNDatasets/subgraph_edges_andxor.csv",
    "GNNDatasets/subgraph_edges_countermux.csv",
    "GNNDatasets/subgraph_edges_fsmor.csv",
]

edges_by_circuit = defaultdict(list)
for ef in edge_files:
    df = pd.read_csv(ef)
    for _, r in df.iterrows():
        edges_by_circuit[r["circuit_name"]].append((r["src"], r["dst"]))

# -----------------------
# Build dataset
# -----------------------
sub_df = pd.read_csv("GNNDatasets/subgraph.csv")
data_list, labels = [], []

for idx, row in tqdm(sub_df.iterrows(), total=len(sub_df), desc="Building subgraphs"):
    ckt = row["circuit_name"]
    lbl = int(row.get("label_subgraph", row.get("label", 0)))
    
    if ckt not in edges_by_circuit:
        continue
    
    # collect node uids from node.csv that belong to this circuit
    sub_nodes = nodes_df[nodes_df["circuit_name"]==ckt]["node"].tolist()
    if not sub_nodes: 
        continue
    
    uid_map = {n:i for i,n in enumerate(sub_nodes)}
    x_list = []
    for n in sub_nodes:
        uid = f"{ckt}::{n}"
        if uid in uid_to_feat:
            x_list.append(uid_to_feat[uid])
        else:
            x_list.append(np.zeros(feat_dim))
    x = torch.tensor(np.vstack(x_list), dtype=torch.float)
    
    # build edge_index
    edge_idx = [[], []]
    for u,v in edges_by_circuit[ckt]:
        if u in uid_map and v in uid_map:
            edge_idx[0].append(uid_map[u]); edge_idx[1].append(uid_map[v])
            edge_idx[0].append(uid_map[v]); edge_idx[1].append(uid_map[u])
    if not edge_idx[0]:
        continue
    edge_index = torch.tensor(edge_idx, dtype=torch.long)
    
    data = Data(x=x, edge_index=edge_index, y=torch.tensor([lbl], dtype=torch.long))
    data.circuit_name = ckt
    data_list.append(data)
    labels.append(lbl)

print(f"Built {len(data_list)} subgraphs (usable)")

# -----------------------
# Split
# -----------------------
labels = np.array(labels)
idxs = np.arange(len(data_list))
train_idx, temp_idx, y_train, y_temp = train_test_split(idxs, labels, test_size=0.3, 
                                                        stratify=labels, random_state=RANDOM_SEED)
val_idx, test_idx, y_val, y_test = train_test_split(temp_idx, y_temp, test_size=0.5,
                                                    stratify=y_temp, random_state=RANDOM_SEED)

train_loader = DataLoader([data_list[i] for i in train_idx], batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader([data_list[i] for i in val_idx], batch_size=BATCH_SIZE, shuffle=False)
test_loader  = DataLoader([data_list[i] for i in test_idx], batch_size=BATCH_SIZE, shuffle=False)

print(f"Train: {len(train_idx)}, Val: {len(val_idx)}, Test: {len(test_idx)}")

# -----------------------
# Model
# -----------------------
class SubgraphClassifier(nn.Module):
    def __init__(self, in_dim, hid_dim=HID_DIM, out_dim=2):
        super().__init__()
        self.conv1 = SAGEConv(in_dim, hid_dim)
        self.conv2 = SAGEConv(hid_dim, hid_dim)
        self.lin = nn.Linear(hid_dim, out_dim)

    def forward(self, x, edge_index, batch):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.4, training=self.training)
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)
        return self.lin(x)

model = SubgraphClassifier(feat_dim).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=5e-4)

# class weights
cls_counts = np.bincount(labels)
w = torch.tensor([cls_counts.sum()/c for c in cls_counts], dtype=torch.float32).to(DEVICE)
criterion = nn.CrossEntropyLoss(weight=w)

# -----------------------
# Training loop
# -----------------------
def evaluate(loader):
    model.eval()
    ys, preds = [], []
    with torch.no_grad():
        for b in loader:
            b = b.to(DEVICE)
            out = model(b.x, b.edge_index, b.batch)
            p = out.argmax(dim=1).cpu().numpy()
            ys.extend(b.y.cpu().numpy())
            preds.extend(p)
    return np.array(ys), np.array(preds)

best_val, best_state = -1, None
for epoch in range(1, EPOCHS+1):
    model.train()
    total_loss=0
    for b in train_loader:
        b = b.to(DEVICE)
        optimizer.zero_grad()
        out = model(b.x, b.edge_index, b.batch)
        loss = criterion(out, b.y.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    if epoch%5==0 or epoch==1:
        yv,pv = evaluate(val_loader)
        acc = accuracy_score(yv,pv)
        print(f"Epoch {epoch:03d} | Loss {total_loss/len(train_loader):.4f} | Val {acc:.4f}")
        if acc>best_val: 
            best_val=acc; best_state=model.state_dict().copy()

# load best
if best_state: model.load_state_dict(best_state)

# -----------------------
# Final test
# -----------------------
yt,pt = evaluate(test_loader)
print("\nFinal Evaluation (Subgraph-Level)")
print("=================================")
print(f"Test Accuracy: {accuracy_score(yt,pt):.4f}\n")
print("Classification Report:")
print(classification_report(yt,pt,digits=4))
print("Confusion Matrix:")
print(confusion_matrix(yt,pt))


/home/rrk307/anaconda3/envs/gnn_circuits/lib/python3.10/site-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /home/rrk307/anaconda3/envs/gnn_circuits/lib/python3.10/site-packages/torch_scatter/_version_cuda.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/home/rrk307/anaconda3/envs/gnn_circuits/lib/python3.10/site-packages/torch_geometric/typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: /home/rrk307/anaconda3/envs/gnn_circuits/lib/python3.10/site-packages/torch_cluster/_version_cuda.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-cluster'. "
/home/rrk307/anaconda3/envs/gnn_circuits/lib/python3.10/site-packages/torch_geometric/typing.py:113: UserWarning: An issue occurred while importing 'torch-spline-

Built 674 subgraphs (usable)
Train: 471, Val: 101, Test: 102
Epoch 001 | Loss 0.7464 | Val 0.2673
Epoch 005 | Loss 0.6193 | Val 0.6238
Epoch 010 | Loss 0.5896 | Val 0.5941
Epoch 015 | Loss 0.5101 | Val 0.6832
Epoch 020 | Loss 0.3784 | Val 0.8020
Epoch 025 | Loss 0.2963 | Val 0.8317
Epoch 030 | Loss 0.2182 | Val 0.8812
Epoch 035 | Loss 0.1549 | Val 0.9208
Epoch 040 | Loss 0.1265 | Val 0.9208
Epoch 045 | Loss 0.1028 | Val 0.9208
Epoch 050 | Loss 0.0897 | Val 0.9208
Epoch 055 | Loss 0.0840 | Val 0.9208
Epoch 060 | Loss 0.0851 | Val 0.9307
Epoch 065 | Loss 0.0641 | Val 0.9307
Epoch 070 | Loss 0.0713 | Val 0.9604
Epoch 075 | Loss 0.0604 | Val 0.9604
Epoch 080 | Loss 0.0645 | Val 0.9604

Final Evaluation (Subgraph-Level)
Test Accuracy: 0.9902

Classification Report:
              precision    recall  f1-score   support

           0     0.9167    1.0000    0.9565        11
           1     1.0000    0.9890    0.9945        91

    accuracy                         0.9902       102
   macro av

#### Jacobain 

In [4]:
# Subgraph-level: PGD-first ? evaluation ? Jacobian + FD relative error
import torch
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support

# ---------------------- PARAMETERS ----------------------
PER_CLASS = 20        # # subgraphs per class to perturb (adjust)
EPSILON = 4.0         # L2 radius for PGD perturbation (on flattened subgraph features)
ALPHA = 1.0           # PGD step size (normalized)
NUM_ITERS = 30        # PGD iterations
FD_EPS = 1e-3         # finite-difference epsilon for Jacobian check
SEED = 42

torch.manual_seed(SEED); np.random.seed(SEED)

# ---------------------- Sanity check ----------------------
required = ["model", "test_loader", "DEVICE"]
for v in required:
    if v not in globals():
        raise RuntimeError(f"Required variable '{v}' not found in the environment.")

model.to(DEVICE)
model.eval()

# ---------------------- Get test dataset as list ----------------------
test_data_list = list(test_loader.dataset)   # preserve the same order used by DataLoader
test_labels = np.array([int(d.y.item()) for d in test_data_list])
n_test = len(test_data_list)
print(f"Test subgraphs: {n_test}; class counts: {np.bincount(test_labels)}")

# ---------------------- Select subgraphs to perturb (PER_CLASS per class) ----------------------
rng = np.random.default_rng(SEED)
selected_idxs = []
for cls in [0, 1]:
    idxs = np.where(test_labels == cls)[0]
    if len(idxs) == 0:
        continue
    k = min(PER_CLASS, len(idxs))
    chosen = rng.choice(idxs, size=k, replace=False)
    selected_idxs.extend(chosen.tolist())
selected_idxs = np.array(sorted(selected_idxs), dtype=np.int64)
print("Selected counts:", {0:int((test_labels[selected_idxs]==0).sum()), 1:int((test_labels[selected_idxs]==1).sum())})

# ---------------------- PGD perturbation for selected subgraphs ----------------------
perturbed_map = {}  # idx -> perturbed x tensor (on DEVICE)
print("\nRunning PGD perturbation on selected subgraphs (this may take a while)...")

for idx in selected_idxs:
    data = test_data_list[int(idx)]
    x_orig = data.x.detach().to(DEVICE)
    edge_index = data.edge_index.to(DEVICE)
    n_nodes, feat_dim = x_orig.shape

    # initialize adv example (random direction scaled to EPSILON)
    delta = torch.randn_like(x_orig, device=DEVICE)
    delta = delta * (EPSILON / (delta.view(-1).norm() + 1e-12))
    x_adv = (x_orig + delta).detach().clone().requires_grad_(True)

    # batch vector for single-graph forward
    batch_zero = torch.zeros(n_nodes, dtype=torch.long, device=DEVICE)

    y_true = torch.tensor([int(data.y.item())], dtype=torch.long, device=DEVICE)

    for it in range(NUM_ITERS):
        # forward
        out = model(x_adv, edge_index, batch_zero)            # shape [1, C]
        loss = F.cross_entropy(out, y_true)
        # gradient wrt inputs
        grad_x = torch.autograd.grad(loss, x_adv, retain_graph=False, create_graph=False)[0]
        gnorm = grad_x.view(-1).norm().item()
        if gnorm == 0:
            break
        step = (ALPHA * grad_x) / (gnorm + 1e-12)
        x_adv = (x_adv + step).detach()
        # project to L2 ball around x_orig
        delta = x_adv - x_orig
        dnorm = delta.view(-1).norm().item()
        if dnorm > EPSILON:
            delta = delta * (EPSILON / (dnorm + 1e-12))
            x_adv = (x_orig + delta).detach()
        x_adv = x_adv.requires_grad_(True)

    perturbed_map[int(idx)] = x_adv.detach().clone()

print("? PGD perturbation finished for selected subgraphs.")

# ---------------------- Evaluate model on full test set (perturbed selected + originals) ----------------------
y_true_list = []
y_pred_list = []

with torch.no_grad():
    for i, data in enumerate(test_data_list):
        x_in = perturbed_map[i] if i in perturbed_map else data.x.to(DEVICE)
        edge_index = data.edge_index.to(DEVICE)
        batch_zero = torch.zeros(x_in.size(0), dtype=torch.long, device=DEVICE)
        out = model(x_in, edge_index, batch_zero)         # [1, C]
        pred = int(out.argmax(dim=1).item())
        y_pred_list.append(pred)
        y_true_list.append(int(data.y.item()))

y_true_arr = np.array(y_true_list)
y_pred_arr = np.array(y_pred_list)

print("\n============= Robustness Evaluation (Full Test Set: perturbed selected + rest original) =============")
acc = (y_true_arr == y_pred_arr).mean()
prec, rec, f1, _ = precision_recall_fscore_support(y_true_arr, y_pred_arr, average='weighted', zero_division=0)
print(f"Accuracy: {acc*100:.2f}%")
print(f"Precision: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}\n")
print("Classification report:")
print(classification_report(y_true_arr, y_pred_arr, labels=[0,1], target_names=['clean','trojan'], digits=4))
print("Confusion Matrix:")
print(confusion_matrix(y_true_arr, y_pred_arr, labels=[0,1]))

# how many selected flipped?
orig_preds = []
adv_preds = []
with torch.no_grad():
    for idx in selected_idxs:
        data = test_data_list[int(idx)]
        edge_index = data.edge_index.to(DEVICE)
        batch_zero = torch.zeros(data.x.size(0), dtype=torch.long, device=DEVICE)
        # original
        out_orig = model(data.x.to(DEVICE), edge_index, batch_zero)
        orig_preds.append(int(out_orig.argmax(dim=1).item()))
        # adv
        out_adv = model(perturbed_map[int(idx)], edge_index, batch_zero)
        adv_preds.append(int(out_adv.argmax(dim=1).item()))
orig_preds = np.array(orig_preds); adv_preds = np.array(adv_preds)
num_flips = int((orig_preds != adv_preds).sum())
print(f"\nSelected subgraphs: {len(selected_idxs)}. Flipped after attack: {num_flips} ({100.0*num_flips/len(selected_idxs):.2f}%).")

# ---------------------- Jacobian & finite-difference relative error (computed AT the PERTURBED input) ----------------------
print("\nComputing Jacobian norms & FD relative error on the PERTURBED selected subgraphs...")
per_sample_info = []   # tuples: (idx, label, jacobian_fro_norm, fd_rel_error)
for idx in selected_idxs:
    data = test_data_list[int(idx)]
    x_adv = perturbed_map[int(idx)].detach().clone().to(DEVICE).requires_grad_(True)  # perturbed input
    edge_index = data.edge_index.to(DEVICE)
    n_nodes, feat_dim = x_adv.shape
    batch_zero = torch.zeros(n_nodes, dtype=torch.long, device=DEVICE)

    # flatten input
    x_flat = x_adv.view(-1).detach().clone().requires_grad_(True)

    def f_flat(x_flat_input):
        x_mat = x_flat_input.view_as(x_adv)
        out = model(x_mat, edge_index, batch_zero)   # [1, C]
        return out.squeeze(0)                        # (C,)

    # Jacobian: shape (C, D) where D = n_nodes * feat_dim
    try:
        J = torch.autograd.functional.jacobian(f_flat, x_flat)    # (C, D)
    except RuntimeError:
        # fallback: compute per-output jacobian rows to avoid memory blowup (slower)
        C = int(model(x_adv, edge_index, batch_zero).shape[1])
        rows = []
        for out_i in range(C):
            def scalar_f(z, i=out_i):
                return f_flat(z)[i]
            r = torch.autograd.functional.jacobian(scalar_f, x_flat)
            rows.append(r.unsqueeze(0))
        J = torch.cat(rows, dim=0)

    J = J.detach()
    jac_frob = float(torch.norm(J, p='fro').item())

    # finite-difference relative error
    delta_fd = FD_EPS * torch.randn_like(x_flat).to(DEVICE)
    pred_change = J @ delta_fd                          # (C,)
    f0 = f_flat(x_flat).detach()
    f0p = f_flat(x_flat + delta_fd).detach()
    actual_change = f0p - f0
    rel_err = float((torch.norm(pred_change - actual_change) / (torch.norm(actual_change) + 1e-8)).item())

    per_sample_info.append((int(idx), int(data.y.item()), jac_frob, rel_err))

# ---------------------- Aggregate & print summary ----------------------
arr = np.array([[i,l,j,r] for (i,l,j,r) in per_sample_info], dtype=object)
if arr.size:
    clean_vals = arr[arr[:,1]==0][:,2].astype(float) if (arr[:,1]==0).any() else np.array([])
    troj_vals  = arr[arr[:,1]==1][:,2].astype(float) if (arr[:,1]==1).any() else np.array([])
    clean_errs = arr[arr[:,1]==0][:,3].astype(float) if (arr[:,1]==0).any() else np.array([])
    troj_errs  = arr[arr[:,1]==1][:,3].astype(float) if (arr[:,1]==1).any() else np.array([])

    def mean_std(a): return (a.mean(), a.std()) if len(a)>0 else (0.0,0.0)

    c_mean, c_std = mean_std(clean_vals)
    t_mean, t_std = mean_std(troj_vals)
    ce_mean, ce_std = mean_std(clean_errs)
    te_mean, te_std = mean_std(troj_errs)

    print("\nJacobian norms & FD relative errors (aggregated):")
    print(f" Clean subgraphs:  avg_norm={c_mean:.4f} ± {c_std:.4f}, avg_FDrel={ce_mean:.4e} ± {ce_std:.4e}")
    print(f" Trojan subgraphs: avg_norm={t_mean:.4f} ± {t_std:.4f}, avg_FDrel={te_mean:.4e} ± {te_std:.4e}")

    print("\nSample preview (first 6): (idx,label,jacobian_frob,FD_rel_err)")
    for t in per_sample_info[:6]:
        print(t)
else:
    print("No Jacobian samples computed (selected set empty).")

print("\nDone. (Order: PGD perturbation -> full-test evaluation -> Jacobian computed at perturbed inputs.)")


Test subgraphs: 102; class counts: [11 91]
Selected counts: {0: 11, 1: 20}

Running PGD perturbation on selected subgraphs (this may take a while)...
? PGD perturbation finished for selected subgraphs.

============= Robustness Evaluation (Full Test Set: perturbed selected + rest original) =============
Accuracy: 96.08%
Precision: 0.9651, Recall: 0.9608, F1: 0.9622

Classification report:
              precision    recall  f1-score   support

       clean     0.7692    0.9091    0.8333        11
      trojan     0.9888    0.9670    0.9778        91

    accuracy                         0.9608       102
   macro avg     0.8790    0.9381    0.9056       102
weighted avg     0.9651    0.9608    0.9622       102

Confusion Matrix:
[[10  1]
 [ 3 88]]

Selected subgraphs: 31. Flipped after attack: 3 (9.68%).

Computing Jacobian norms & FD relative error on the PERTURBED selected subgraphs...

Jacobian norms & FD relative errors (aggregated):
 Clean subgraphs:  avg_norm=0.4876 ± 0.1973, avg_F

#### Local Lipschitz Constants

In [6]:
# ----------------------- Local Lipschitz (Subgraph-Level) -----------------------
import torch, numpy as np, torch.nn.functional as F
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support

# ----- parameters (tune if needed) -----
PER_CLASS = 20      # up to this many per class from test set (min with available)
FD_EPS = 1e-3
EPSILON = 5.0       # L2 budget for PGD (feature units)
ALPHA = 1.0         # PGD step size
NUM_ITERS = 30      # PGD iters
SEED = 42

torch.manual_seed(SEED); np.random.seed(SEED)

# ----- quick sanity -----
if 'model' not in globals() or 'test_loader' not in globals():
    raise RuntimeError("Required variables `model` and `test_loader` must exist in the notebook.")

model.to(DEVICE)
model.eval()

# ----- get dataset (robust to DataLoader type) -----
dataset = test_loader.dataset
n_test = len(dataset)

# extract labels array
labels_np = np.array([int(d.y.item() if hasattr(d.y, 'item') else d.y[0].item()) for d in dataset])

# ----- select indices (PER_CLASS per class) -----
rng = np.random.default_rng(SEED)
selected = []
for cls in [0, 1]:
    idxs = np.where(labels_np == cls)[0].tolist()
    if len(idxs) == 0:
        continue
    k = min(PER_CLASS, len(idxs))
    chosen = rng.choice(idxs, size=k, replace=False).tolist()
    selected.extend(chosen)
selected = np.array(selected, dtype=np.int64)
print("Selected perturbation pool:", {0:int((labels_np[selected]==0).sum()), 1:int((labels_np[selected]==1).sum())})

# ----- perturb each selected subgraph using top-right singular vector init + PGD -----
perturbed_map = {}   # idx -> perturbed_x (on DEVICE)
orig_preds_list = []
adv_preds_list = []

print("\nRunning Lipschitz-directed PGD for selected subgraphs (this may take a while)...")
for idx in selected:
    data = dataset[int(idx)]
    x_orig = data.x.detach().clone().to(DEVICE)            # (num_nodes, feat_dim)
    edge_index = data.edge_index.to(DEVICE)
    y_true = torch.tensor([int(data.y.item())], dtype=torch.long, device=DEVICE)

    # helper: produce batch vector of zeros for single graph
    def batch_for(x):
        return torch.zeros(x.size(0), dtype=torch.long, device=DEVICE)

    # define f_local that accepts x (num_nodes,feat) and returns logits (num_classes,)
    def f_local(x):
        return model(x, edge_index, batch_for(x)).squeeze(0)

    # Compute Jacobian at x_orig (shape: (num_classes, num_nodes, feat_dim))
    # NOTE: we compute Jacobian once per graph to get singular vector init
    try:
        J = torch.autograd.functional.jacobian(f_local, x_orig)    # (C, N, F)
    except RuntimeError:
        # if autograd fails due to graph size, compute per-output jac manually (slower)
        logits0 = f_local(x_orig).detach()
        C = logits0.shape[0]
        rows = []
        for c in range(C):
            def scalar_f(x, cidx=c):
                return f_local(x)[cidx]
            row = torch.autograd.functional.jacobian(scalar_f, x_orig)  # (N, F)
            rows.append(row.unsqueeze(0))
        J = torch.cat(rows, dim=0)  # (C, N, F)

    # flatten J to shape (C, d) where d = N * F
    C = J.shape[0]
    d = int(J.shape[1] * J.shape[2])
    J_flat = J.reshape(C, d)

    # get top-right singular vector v (length d)
    try:
        _, S, Vh = torch.linalg.svd(J_flat, full_matrices=False)
        v = Vh[0, :].detach().to(DEVICE)   # (d,)
    except RuntimeError:
        # fallback random direction
        v = torch.randn(d, device=DEVICE)
    if v.norm().item() > 0:
        v = v / (v.norm() + 1e-12)
    else:
        v = torch.randn_like(v).to(DEVICE); v = v / (v.norm() + 1e-12)

    # initialize adversarial x_adv (reshape v -> (N,F))
    v_mat = v.view(x_orig.shape[0], x_orig.shape[1])
    x_adv = (x_orig + 0.5 * EPSILON * v_mat).detach().clone().requires_grad_(True)

    # PGD loop maximize CE loss (untargeted)
    for it in range(NUM_ITERS):
        logits = model(x_adv, edge_index, batch_for(x_adv))
        loss = F.cross_entropy(logits, y_true)
        grad = torch.autograd.grad(loss, x_adv, retain_graph=False, create_graph=False)[0]
        gn = grad.view(-1)
        gnorm = gn.norm().item()
        if gnorm == 0:
            break
        step = ALPHA * (grad / (gnorm + 1e-12))
        x_adv = (x_adv + step).detach()
        # project to L2-ball around x_orig
        delta = (x_adv - x_orig).view(-1)
        dnorm = delta.norm().item()
        if dnorm > EPSILON:
            delta = delta * (EPSILON / (dnorm + 1e-12))
            x_adv = (x_orig + delta.view_as(x_orig)).detach()
        x_adv = x_adv.requires_grad_(True)

    # finalize
    x_adv_final = x_adv.detach()
    perturbed_map[int(idx)] = x_adv_final

    # store preds for flip statistics
    with torch.no_grad():
        p_orig = f_local(x_orig).argmax().item()
        p_adv  = f_local(x_adv_final).argmax().item()
    orig_preds_list.append(p_orig)
    adv_preds_list.append(p_adv)

print("? Finished perturbations.")

# ----- Evaluate full test set using perturbed subgraphs for selected indices -----
print("\n============= Robustness Evaluation (Full Test Set) =============")
preds_all = []
labels_all = []
with torch.no_grad():
    for i, data in enumerate(dataset):
        x = data.x.detach().clone().to(DEVICE)
        edge_index = data.edge_index.to(DEVICE)
        lab = int(data.y.item())
        if int(i) in perturbed_map:
            x_eval = perturbed_map[int(i)]
        else:
            x_eval = x
        logits = model(x_eval, edge_index, batch_for(x_eval))
        preds_all.append(int(logits.argmax(dim=1).item()))
        labels_all.append(lab)

preds_all = np.array(preds_all)
labels_all = np.array(labels_all)

acc = (preds_all == labels_all).mean()
prec, rec, f1, _ = precision_recall_fscore_support(labels_all, preds_all, average='weighted', zero_division=0)
print(f"Accuracy: {acc*100:.2f}%")
print(f"Precision: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}\n")
print("Classification Report:")
print(classification_report(labels_all, preds_all, target_names=['clean','trojan'], digits=4))
print("Confusion Matrix:")
print(confusion_matrix(labels_all, preds_all, labels=[0,1]))

# flip statistics
num_flips = int((np.array(orig_preds_list) != np.array(adv_preds_list)).sum())
print(f"\nSelected subgraphs: {len(selected)}. Flipped after attack: {num_flips} ({100.0*num_flips/len(selected):.2f}%).")

# ----- Now compute Local Lipschitz constants and FD relative error ON THE PERTURBED SUBGRAPHS -----
print("\nComputing Local Lipschitz constants + FD relative errors (on perturbed subgraphs)...")
per_sample_info = []   # tuples (dataset_idx, label, sigma_max, fd_rel_err)
for i, idx in enumerate(selected):
    data = dataset[int(idx)]
    x_adv = perturbed_map[int(idx)].detach().clone().to(DEVICE).requires_grad_(True)
    edge_index = data.edge_index.to(DEVICE)
    label = int(data.y.item())

    # define f_local on perturbed input
    def f_local_pert(x):
        return model(x, edge_index, batch_for(x)).squeeze(0)

    # jacobian shape (C, N, F)
    try:
        J = torch.autograd.functional.jacobian(f_local_pert, x_adv).detach()   # (C, N, F)
    except RuntimeError:
        # fallback per-output jac (slower)
        logits0 = f_local_pert(x_adv).detach()
        C = logits0.shape[0]
        rows = []
        for c in range(C):
            def scalar_f(x, cidx=c):
                return f_local_pert(x)[cidx]
            row = torch.autograd.functional.jacobian(scalar_f, x_adv)
            rows.append(row.unsqueeze(0))
        J = torch.cat(rows, dim=0)

    C = J.shape[0]
    d = int(J.shape[1] * J.shape[2])
    J_flat = J.reshape(C, d)

    # spectral norm via SVD (largest singular value)
    try:
        _, S, _ = torch.linalg.svd(J_flat, full_matrices=False)
        sigma_max = float(S[0].item())
    except RuntimeError:
        # fallback via eigen on JJT
        JJT = (J_flat @ J_flat.T).cpu().numpy()
        eigvals = np.linalg.eigvalsh(JJT)
        sigma_max = float(np.sqrt(max(eigvals.max(), 0.0)))

    # finite-difference relative error
    delta_fd = FD_EPS * torch.randn_like(x_adv).to(DEVICE).view(-1)   # length d
    pred_change = (J_flat @ delta_fd)                 # (C,)
    f0 = f_local_pert(x_adv).detach()
    f0p = f_local_pert((x_adv + delta_fd.view_as(x_adv))).detach()
    actual_change = f0p - f0
    fd_rel_err = (torch.norm(pred_change - actual_change) / (torch.norm(actual_change) + 1e-8)).item()

    per_sample_info.append((int(idx), label, float(sigma_max), float(fd_rel_err)))

# aggregate and print
clean_stats = [p for p in per_sample_info if p[1] == 0]
troj_stats  = [p for p in per_sample_info if p[1] == 1]

def aggs(stats):
    if not stats:
        return (0.0, 0.0, 0.0, 0.0)
    Ls = np.array([s[2] for s in stats])
    Es = np.array([s[3] for s in stats])
    return (Ls.mean(), Ls.std(), Es.mean(), Es.std())

cL_mean, cL_std, cE_mean, cE_std = aggs(clean_stats)
tL_mean, tL_std, tE_mean, tE_std = aggs(troj_stats)

print("\n--- Local Lipschitz Constants (on perturbed subgraphs) ---")
print(f" Clean:  avg_L={cL_mean:.4f} ± {cL_std:.4f}, avg_FDrel={cE_mean:.4e} ± {cE_std:.4e}")
print(f" Trojan: avg_L={tL_mean:.4f} ± {tL_std:.4f}, avg_FDrel={tE_mean:.4e} ± {tE_std:.4e}")

print("\nSample preview (first 6): (idx,label,L,FD_rel_err)")
for p in per_sample_info[:6]:
    print(p)


Selected perturbation pool: {0: 11, 1: 20}

Running Lipschitz-directed PGD for selected subgraphs (this may take a while)...
? Finished perturbations.

============= Robustness Evaluation (Full Test Set) =============
Accuracy: 96.08%
Precision: 0.9651, Recall: 0.9608, F1: 0.9622

Classification Report:
              precision    recall  f1-score   support

       clean     0.7692    0.9091    0.8333        11
      trojan     0.9888    0.9670    0.9778        91

    accuracy                         0.9608       102
   macro avg     0.8790    0.9381    0.9056       102
weighted avg     0.9651    0.9608    0.9622       102

Confusion Matrix:
[[10  1]
 [ 3 88]]

Selected subgraphs: 31. Flipped after attack: 3 (9.68%).

Computing Local Lipschitz constants + FD relative errors (on perturbed subgraphs)...

--- Local Lipschitz Constants (on perturbed subgraphs) ---
 Clean:  avg_L=0.4934 ± 0.2302, avg_FDrel=1.4172e-02 ± 1.3608e-02
 Trojan: avg_L=1.0035 ± 2.6309, avg_FDrel=7.5908e-01 ± 2.5349

#### Hessian-based Curvature Analysis

In [8]:
# ----------------------- Hessian-Based Curvature (Subgraph-Level) -----------------------
import torch, numpy as np, torch.nn.functional as F
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support

# ----- parameters -----
PER_CLASS = 20        # subgraphs per class
FD_EPS = 5e-3         # finite diff epsilon
TRIALS_PER_GRAPH = 5  # #trials for FD error
PERT_P = 10.0          # L2 magnitude for Hessian-aligned perturbation
SEED = 42

torch.manual_seed(SEED); np.random.seed(SEED)

if 'model' not in globals() or 'test_loader' not in globals():
    raise RuntimeError("Need `model` and `test_loader` in environment.")

model.to(DEVICE)
model.eval()

dataset = test_loader.dataset
n_test = len(dataset)
labels_np = np.array([int(d.y.item()) for d in dataset])

# ----- select PER_CLASS subgraphs per class -----
rng = np.random.default_rng(SEED)
selected = []
for cls in [0, 1]:
    idxs = np.where(labels_np == cls)[0].tolist()
    k = min(PER_CLASS, len(idxs))
    chosen = rng.choice(idxs, size=k, replace=False).tolist()
    selected.extend(chosen)
selected = np.array(selected, dtype=np.int64)
print(f"Selected pool: clean={int((labels_np[selected]==0).sum())}, trojan={int((labels_np[selected]==1).sum())}")

# ----- base predictions -----
def batch_for(x): return torch.zeros(x.size(0), dtype=torch.long, device=DEVICE)
with torch.no_grad():
    base_preds = []
    for data in dataset:
        logits = model(data.x.to(DEVICE), data.edge_index.to(DEVICE), batch_for(data.x.to(DEVICE)))
        base_preds.append(int(logits.argmax()))
base_preds = np.array(base_preds)

# ----- compute gradient norms + FD relative errors -----
per_sample_info = []  # (idx, label, lambda_max, avg_rel_err)

print("\nComputing Hessian curvature proxies on selected subgraphs...")
for idx in selected:
    data = dataset[int(idx)]
    x0 = data.x.detach().clone().to(DEVICE).requires_grad_(True)
    edge_index = data.edge_index.to(DEVICE)

    # predicted class at x0
    with torch.no_grad():
        logits = model(x0, edge_index, batch_for(x0))
    pred_class = int(logits.argmax().item())

    def h(x):
        return F.log_softmax(model(x, edge_index, batch_for(x)).squeeze(0), dim=0)[pred_class]

    h0 = h(x0)
    g = torch.autograd.grad(h0, x0, retain_graph=False, create_graph=False)[0].detach()

    lambda_max = float(g.norm(p=2).item() ** 2)

    # FD relative error
    errs = []
    for _ in range(TRIALS_PER_GRAPH):
        delta = FD_EPS * torch.randn_like(x0).to(DEVICE)
        gt_delta = float((g * delta).sum().item())
        pred_second = 0.5 * (gt_delta ** 2)
        actual_second = float((h(x0 + delta) - h0 - (g * delta).sum()).item())
        rel_err = abs(pred_second - actual_second) / (abs(actual_second) + 1e-8)
        errs.append(rel_err)
    avg_rel_err = float(np.mean(errs))

    per_sample_info.append((int(idx), int(data.y.item()), lambda_max, avg_rel_err))

# ----- build Hessian-aligned perturbations -----
print("\nConstructing Hessian-aligned perturbations...")
perturbed_map = {}
for (idx, label, lambda_val, avg_rel_err) in per_sample_info:
    data = dataset[int(idx)]
    x0 = data.x.detach().clone().to(DEVICE).requires_grad_(True)
    edge_index = data.edge_index.to(DEVICE)

    def h(x):
        return F.log_softmax(model(x, edge_index, batch_for(x)).squeeze(0), dim=0)[int(data.y.item())]

    g = torch.autograd.grad(h(x0), x0, retain_graph=False, create_graph=False)[0].detach()
    gnorm = g.norm().item()
    if gnorm < 1e-12:
        dir_vec = torch.randn_like(x0).to(DEVICE)
    else:
        dir_vec = - g / (gnorm + 1e-12)

    delta = (PERT_P * dir_vec).detach()
    perturbed_map[int(idx)] = (x0 + delta).detach()

# ----- evaluate on full test set -----
print("\n================ Robustness Evaluation (Full Test Set: perturbed+clean) ===============")
preds_all, labels_all = [], []
with torch.no_grad():
    for i, data in enumerate(dataset):
        x = data.x.to(DEVICE)
        if int(i) in perturbed_map:
            x_eval = perturbed_map[int(i)]
        else:
            x_eval = x
        logits = model(x_eval, data.edge_index.to(DEVICE), batch_for(x_eval))
        preds_all.append(int(logits.argmax()))
        labels_all.append(int(data.y.item()))

preds_all = np.array(preds_all)
labels_all = np.array(labels_all)

acc = (preds_all == labels_all).mean()
prec, rec, f1, _ = precision_recall_fscore_support(labels_all, preds_all, average='weighted', zero_division=0)
print(f"Accuracy: {acc*100:.2f}%")
print(f"Precision: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}\n")
print("Classification Report:")
print(classification_report(labels_all, preds_all, target_names=['clean','trojan'], digits=4))
print("Confusion Matrix:")
print(confusion_matrix(labels_all, preds_all, labels=[0,1]))

# ----- flip stats -----
orig_sel_preds = base_preds[selected]
adv_sel_preds = np.array([preds_all[i] for i in selected])
num_flips = int((orig_sel_preds != adv_sel_preds).sum())
print(f"\nSelected subgraphs: {len(selected)}. Flipped after perturbation: {num_flips} ({100.0*num_flips/len(selected):.2f}%).")

# ----- aggregate curvature stats -----
clean_stats = [p for p in per_sample_info if p[1] == 0]
troj_stats  = [p for p in per_sample_info if p[1] == 1]

def summarize(stats):
    if not stats: return (0.0,0.0,0.0,0.0)
    Ls = np.array([s[2] for s in stats])
    Es = np.array([s[3] for s in stats])
    return (Ls.mean(), Ls.std(), Es.mean(), Es.std())

cL_mean, cL_std, cE_mean, cE_std = summarize(clean_stats)
tL_mean, tL_std, tE_mean, tE_std = summarize(troj_stats)

print("\n--- Hessian Curvature Stats (grad outer-product proxy) ---")
print(f" Clean:  avg_lambda={cL_mean:.4f} ± {cL_std:.4f}, avg_FDrel={cE_mean:.4e} ± {cE_std:.4e}")
print(f" Trojan: avg_lambda={tL_mean:.4f} ± {tL_std:.4f}, avg_FDrel={tE_mean:.4e} ± {tE_std:.4e}")

print("\nSample preview (first 6): (idx,label,lambda,FD_rel_err)")
for p in per_sample_info[:6]:
    print(p)


Selected pool: clean=11, trojan=20

Computing Hessian curvature proxies on selected subgraphs...

Constructing Hessian-aligned perturbations...

================ Robustness Evaluation (Full Test Set: perturbed+clean) ===============
Accuracy: 93.14%
Precision: 0.9289, Recall: 0.9314, F1: 0.9299

Classification Report:
              precision    recall  f1-score   support

       clean     0.7000    0.6364    0.6667        11
      trojan     0.9565    0.9670    0.9617        91

    accuracy                         0.9314       102
   macro avg     0.8283    0.8017    0.8142       102
weighted avg     0.9289    0.9314    0.9299       102

Confusion Matrix:
[[ 7  4]
 [ 3 88]]

Selected subgraphs: 31. Flipped after perturbation: 6 (19.35%).

--- Hessian Curvature Stats (grad outer-product proxy) ---
 Clean:  avg_lambda=0.0001 ± 0.0003, avg_FDrel=9.4980e-01 ± 3.6951e-02
 Trojan: avg_lambda=0.0001 ± 0.0003, avg_FDrel=5.0204e-01 ± 4.5488e-01

Sample preview (first 6): (idx,label,lambda,FD_r

#### Prediction Margin

In [14]:
# ----------------------- Prediction Margin (Subgraph-Level) -----------------------
import torch, numpy as np, torch.nn.functional as F
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support

# ----- parameters -----
PER_CLASS = 20        # subgraphs per class
EPSILON = 5.0         # perturbation budget (strong)
ALPHA = 0.4           # PGD step size
NUM_ITERS = 15        # PGD iterations
FD_EPS = 1e-3         # finite-difference perturbation
SEED = 42

torch.manual_seed(SEED); np.random.seed(SEED)

if 'model' not in globals() or 'test_loader' not in globals():
    raise RuntimeError("Need `model` and `test_loader` in environment.")

model.to(DEVICE)
model.eval()

dataset = test_loader.dataset
n_test = len(dataset)
labels_np = np.array([int(d.y.item()) for d in dataset])

# ----- select PER_CLASS subgraphs per class -----
rng = np.random.default_rng(SEED)
selected = []
for cls in [0, 1]:
    idxs = np.where(labels_np == cls)[0].tolist()
    k = min(PER_CLASS, len(idxs))
    chosen = rng.choice(idxs, size=k, replace=False).tolist()
    selected.extend(chosen)
selected = np.array(selected, dtype=np.int64)
print(f"Selected pool: clean={int((labels_np[selected]==0).sum())}, trojan={int((labels_np[selected]==1).sum())}")

# helper for batching
def batch_for(x): 
    return torch.zeros(x.size(0), dtype=torch.long, device=DEVICE)

# ----- base predictions -----
with torch.no_grad():
    base_preds = []
    for data in dataset:
        logits = model(data.x.to(DEVICE), data.edge_index.to(DEVICE), batch_for(data.x.to(DEVICE)))
        base_preds.append(int(logits.argmax()))
base_preds = np.array(base_preds)

# ----- adversarial perturbations (PGD) -----
perturbed_map = {}

print("\nRunning PGD perturbations on selected subgraphs...")
for idx in selected:
    data = dataset[int(idx)]
    x_orig = data.x.detach().clone().to(DEVICE)
    edge_index = data.edge_index.to(DEVICE)
    y_scalar = int(data.y.item())
    target = torch.tensor([y_scalar], dtype=torch.long, device=DEVICE)  # shape (1,)

    # random init inside epsilon-ball
    delta = torch.randn_like(x_orig).to(DEVICE)
    delta = EPSILON * delta / (delta.norm() + 1e-12)
    x_adv = (x_orig + delta).detach().clone().requires_grad_(True)

    for it in range(NUM_ITERS):
        logits = model(x_adv, edge_index, batch_for(x_adv))  # shape (1, C)
        loss = F.cross_entropy(logits, target)               # target shape (1,)
        grad = torch.autograd.grad(loss, x_adv, retain_graph=False, create_graph=False)[0]

        step = ALPHA * grad / (grad.norm() + 1e-12)
        x_adv = (x_adv + step).detach()
        delta = x_adv - x_orig
        if delta.norm() > EPSILON:
            delta = delta * (EPSILON / (delta.norm() + 1e-12))
            x_adv = (x_orig + delta).detach()
        x_adv = x_adv.requires_grad_(True)

    perturbed_map[int(idx)] = x_adv.detach()

print("? Finished perturbations.")

# ----- evaluate on full test set (perturbed + clean) -----
print("\n================ Robustness Evaluation (Full Test Set: perturbed+clean) ===============")
preds_all, labels_all = [], []
with torch.no_grad():
    for i, data in enumerate(dataset):
        x = data.x.to(DEVICE)
        if int(i) in perturbed_map:
            x_eval = perturbed_map[int(i)]
        else:
            x_eval = x
        logits = model(x_eval, data.edge_index.to(DEVICE), batch_for(x_eval))
        preds_all.append(int(logits.argmax()))
        labels_all.append(int(data.y.item()))

preds_all = np.array(preds_all)
labels_all = np.array(labels_all)

acc = (preds_all == labels_all).mean()
prec, rec, f1, _ = precision_recall_fscore_support(labels_all, preds_all, average='weighted', zero_division=0)
print(f"Accuracy: {acc*100:.2f}%")
print(f"Precision: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}\n")
print("Classification Report:")
print(classification_report(labels_all, preds_all, target_names=['clean','trojan'], digits=4))
print("Confusion Matrix:")
print(confusion_matrix(labels_all, preds_all, labels=[0,1]))

# ----- flip stats -----
orig_sel_preds = base_preds[selected]
adv_sel_preds = np.array([preds_all[i] for i in selected])
num_flips = int((orig_sel_preds != adv_sel_preds).sum())
print(f"\nSelected subgraphs: {len(selected)}. Flipped after perturbation: {num_flips} ({100.0*num_flips/len(selected):.2f}%).")

# ----- compute prediction margin + FD relative error -----
per_sample_info = []  # (idx, label, margin, FD_rel_err)
for idx in selected:
    data = dataset[int(idx)]
    x_eval = perturbed_map[int(idx)]
    edge_index = data.edge_index.to(DEVICE)

    with torch.no_grad():
        logits = model(x_eval, edge_index, batch_for(x_eval)).squeeze(0)
    pred_class = int(logits.argmax().item())
    pred_logit = logits[pred_class].item()
    other_logits = logits.clone()
    other_logits[pred_class] = -float('inf')
    second_max = other_logits.max().item()
    margin = pred_logit - second_max

    # finite difference check
    delta = FD_EPS * torch.randn_like(x_eval).to(DEVICE)
    with torch.no_grad():
        logits_p = model(x_eval + delta, edge_index, batch_for(x_eval)).squeeze(0)
    pred_logit_p = logits_p[pred_class].item()
    other_logits_p = logits_p.clone()
    other_logits_p[pred_class] = -float('inf')
    second_max_p = other_logits_p.max().item()
    margin_p = pred_logit_p - second_max_p

    rel_err = abs(margin - margin_p) / (abs(margin_p) + 1e-12)
    per_sample_info.append((int(idx), int(data.y.item()), float(margin), float(rel_err)))

# ----- aggregate stats -----
clean_stats = [p for p in per_sample_info if p[1] == 0]
troj_stats  = [p for p in per_sample_info if p[1] == 1]

def summarize(stats):
    if not stats: return (0.0,0.0,0.0,0.0)
    Ms = np.array([s[2] for s in stats])
    Es = np.array([s[3] for s in stats])
    return (Ms.mean(), Ms.std(), Es.mean(), Es.std())

cM_mean, cM_std, cE_mean, cE_std = summarize(clean_stats)
tM_mean, tM_std, tE_mean, tE_std = summarize(troj_stats)

print("\n--- Prediction Margin Stats (on perturbed subgraphs) ---")
print(f" Clean:  avg_margin={cM_mean:.4f} ± {cM_std:.4f}, avg_FDrel={cE_mean:.4e} ± {cE_std:.4e}")
print(f" Trojan: avg_margin={tM_mean:.4f} ± {tM_std:.4f}, avg_FDrel={tE_mean:.4e} ± {tE_std:.4e}")

print("\nSample preview (first 6): (idx,label,margin,FD_rel_err)")
for p in per_sample_info[:6]:
    print(p)


Selected pool: clean=11, trojan=20

Running PGD perturbations on selected subgraphs...
? Finished perturbations.

================ Robustness Evaluation (Full Test Set: perturbed+clean) ===============
Accuracy: 96.08%
Precision: 0.9651, Recall: 0.9608, F1: 0.9622

Classification Report:
              precision    recall  f1-score   support

       clean     0.7692    0.9091    0.8333        11
      trojan     0.9888    0.9670    0.9778        91

    accuracy                         0.9608       102
   macro avg     0.8790    0.9381    0.9056       102
weighted avg     0.9651    0.9608    0.9622       102

Confusion Matrix:
[[10  1]
 [ 3 88]]

Selected subgraphs: 31. Flipped after perturbation: 3 (9.68%).

--- Prediction Margin Stats (on perturbed subgraphs) ---
 Clean:  avg_margin=2.8802 ± 0.9243, avg_FDrel=2.6496e-04 ± 4.0346e-04
 Trojan: avg_margin=88.2754 ± 267.9658, avg_FDrel=6.3873e-05 ± 8.6440e-05

Sample preview (first 6): (idx,label,margin,FD_rel_err)
(14, 0, 3.2770175933837

#### Adversarial Robustness Radius

In [17]:
# ----------------------- Adversarial Robustness Radius (Subgraph-Level) -----------------------
import time

# ----- helpers -----
def f_for_subgraph(x_tensor, data):
    """Return logits for given subgraph with node features replaced by x_tensor."""
    x_tensor = x_tensor.to(DEVICE)
    with torch.no_grad():
        out = model(x_tensor, data.edge_index.to(DEVICE), batch_for(x_tensor))
    return out.squeeze(0)

def adversarial_radius_for_subgraph(data, x0, initial_epsilon=1e-3, growth_factor=1.25,
                                    max_epsilon=20.0, bs_iters=10, num_trials=6):
    """Estimate minimal perturbation radius that flips prediction (around perturbed point)."""
    with torch.no_grad():
        base_out = model(x0, data.edge_index.to(DEVICE), batch_for(x0))
        y0 = int(base_out.argmax().item())

    def is_same(x):
        out = f_for_subgraph(x, data)
        return int(out.argmax().item()) == y0

    radii = []
    for _ in range(num_trials):
        d = torch.randn_like(x0)
        d = d / (d.norm() + 1e-12)

        eps = initial_epsilon
        while eps < max_epsilon and is_same(x0 + eps * d):
            eps *= growth_factor

        if eps >= max_epsilon:
            candidate = max_epsilon
        else:
            low, high = eps / growth_factor, eps
            for _ in range(bs_iters):
                mid = 0.5 * (low + high)
                if is_same(x0 + mid * d):
                    low = mid
                else:
                    high = mid
            candidate = float(high)
        radii.append(candidate)
    return float(min(radii))

def adversarial_radius_relerr(data, x0):
    r1 = adversarial_radius_for_subgraph(data, x0, growth_factor=2.25, bs_iters=10, num_trials=6)
    r2 = adversarial_radius_for_subgraph(data, x0, growth_factor=1.4, bs_iters=12, num_trials=6)
    rel_err = abs(r1 - r2) / (abs(r2) + 1e-12)
    return r1, rel_err

# ----- ARR computation on selected perturbed subgraphs -----
class_names = ['clean', 'trojan']
class_adv_radius = {cn: [] for cn in class_names}
class_rel_errors = {cn: [] for cn in class_names}
all_radii, all_rel_errs = [], []

t0 = time.time()
print("\nComputing Adversarial Robustness Radius (ARR) for selected perturbed subgraphs...")
for i, idx in enumerate(selected):
    idx = int(idx)
    data = dataset[idx]
    label = int(data.y.item())
    cn = class_names[label]

    x0 = perturbed_map[idx].clone().detach().to(DEVICE)  # start from perturbed features
    r, rel = adversarial_radius_relerr(data, x0)

    class_adv_radius[cn].append(r)
    class_rel_errors[cn].append(rel)
    all_radii.append(r)
    all_rel_errs.append(rel)

    if (i+1) % 20 == 0:
        print(f"  processed {i+1}/{len(selected)} subgraphs...")

t1 = time.time()
print(f"? Done ARR computation. Time elapsed: {t1-t0:.1f}s")

# ----- Reporting ARR aggregates -----
print("\n--- Adversarial Robustness Radius (ARR) Stats ---")
print("{:<10s} {:>14s} {:>22s}".format("Class", "Avg Radius ± Std", "Avg Rel. Error ± Std"))
print("-"*52)
for cn in class_names:
    if class_adv_radius[cn]:
        print("{:<10s} {:>7.4f} ± {:<7.4f} {:>14.4e} ± {:<10.4e}".format(
            cn, np.mean(class_adv_radius[cn]), np.std(class_adv_radius[cn]),
            np.mean(class_rel_errors[cn]), np.std(class_rel_errors[cn])
        ))
    else:
        print("{:<10s} {:>10s}".format(cn, "-"))

print("\nOverall ARR: Avg Radius: {:.4f} ± {:.4f}".format(np.mean(all_radii), np.std(all_radii)))
print("Overall ARR: Avg Relative Error: {:.4e} ± {:.4e}".format(np.mean(all_rel_errs), np.std(all_rel_errs)))



Computing Adversarial Robustness Radius (ARR) for selected perturbed subgraphs...
  processed 20/31 subgraphs...
? Done ARR computation. Time elapsed: 23.6s

--- Adversarial Robustness Radius (ARR) Stats ---
Class      Avg Radius ± Std   Avg Rel. Error ± Std
----------------------------------------------------
clean      20.0000 ± 0.0000      0.0000e+00 ± 0.0000e+00
trojan     20.0000 ± 0.0000      0.0000e+00 ± 0.0000e+00

Overall ARR: Avg Radius: 20.0000 ± 0.0000
Overall ARR: Avg Relative Error: 0.0000e+00 ± 0.0000e+00


In [18]:
# ------------------ Evaluate model on full test set (200 perturbed + rest unperturbed) ------------------
model.eval()
with torch.no_grad():
    all_logits = []
    all_labels = []
    for i, data in enumerate(dataset):
        x_in = perturbed_map[i].to(DEVICE) if i in perturbed_map else data.x.to(DEVICE)
        logits = model(x_in, data.edge_index.to(DEVICE), batch_for(x_in))
        all_logits.append(logits.cpu())
        all_labels.append(int(data.y.item()))

    all_logits = torch.cat(all_logits, dim=0)
    preds_all = all_logits.argmax(dim=1).numpy()
    labels_all = np.array(all_labels)

acc = (preds_all == labels_all).mean()
prec, rec, f1, _ = precision_recall_fscore_support(labels_all, preds_all, average='weighted', zero_division=0)

print("\n============= Robustness Evaluation (Full Test Set: perturbed selected subgraphs + others unperturbed) =============")
print(f"Accuracy: {acc*100:.2f}%")
print(f"Precision: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}\n")
print("Classification report:")
print(classification_report(labels_all, preds_all, target_names=['clean','trojan'], digits=4))
print("Confusion Matrix:")
print(confusion_matrix(labels_all, preds_all, labels=[0,1]))

# ------------------ Flip statistics for selected subgraphs ------------------
with torch.no_grad():
    orig_preds = []
    for data in dataset:
        x_in = data.x.to(DEVICE)
        logits = model(x_in, data.edge_index.to(DEVICE), batch_for(x_in))
        orig_preds.append(logits.cpu())
    orig_preds = torch.cat(orig_preds, dim=0).argmax(dim=1).numpy()

adv_sel_preds = preds_all[selected]
num_flips = int((orig_preds[selected] != adv_sel_preds).sum())
print(f"\nSelected subgraphs: {len(selected)}. Flipped after perturbation: {num_flips} ({100.0 * num_flips/len(selected):.2f}%).")



============= Robustness Evaluation (Full Test Set: perturbed selected subgraphs + others unperturbed) =============
Accuracy: 96.08%
Precision: 0.9651, Recall: 0.9608, F1: 0.9622

Classification report:
              precision    recall  f1-score   support

       clean     0.7692    0.9091    0.8333        11
      trojan     0.9888    0.9670    0.9778        91

    accuracy                         0.9608       102
   macro avg     0.8790    0.9381    0.9056       102
weighted avg     0.9651    0.9608    0.9622       102

Confusion Matrix:
[[10  1]
 [ 3 88]]

Selected subgraphs: 31. Flipped after perturbation: 3 (9.68%).


#### Stability Under Input Noise

In [20]:
# ============================================================
# Stability Under Input Noise (SUIN) - Subgraph classification
# ============================================================
import torch
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support
import time

# --- Parameters ---
NOISE_SIGMA        = 0.5   # Gaussian noise stddev for stability metric
NUM_NOISE_SAMPLES  = 20     # Monte Carlo samples per subgraph
RELERR_RESAMPLES   = 5      # repeats to estimate relative error

print("\n--- Stability Under Input Noise (PGD-first, then metric) ---")
print(f"Selected subgraphs: clean={(labels_np[selected]==0).sum()}, trojan={(labels_np[selected]==1).sum()}")

# ============================================================
# Step 1: Evaluate model on full dataset (perturbed + original)
# ============================================================
model.eval()
with torch.no_grad():
    all_logits, all_labels = [], []
    for i, data in enumerate(dataset):
        x_in = perturbed_map[i].to(DEVICE) if i in perturbed_map else data.x.to(DEVICE)
        logits = model(x_in, data.edge_index.to(DEVICE), batch_for(x_in))
        all_logits.append(logits.cpu())
        all_labels.append(int(data.y.item()))

    all_logits = torch.cat(all_logits, dim=0)
    preds_all  = all_logits.argmax(dim=1).numpy()
    labels_all = np.array(all_labels)

acc = (preds_all == labels_all).mean()
prec, rec, f1, _ = precision_recall_fscore_support(labels_all, preds_all, average='weighted', zero_division=0)

print("\n============= Robustness Evaluation (Full Test Set: perturbed selected + others unperturbed) =============")
print(f"Accuracy: {acc*100:.2f}%")
print(f"Precision: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}\n")
print("Classification report:")
print(classification_report(labels_all, preds_all, target_names=['clean','trojan'], digits=4))
print("Confusion Matrix:")
print(confusion_matrix(labels_all, preds_all, labels=[0,1]))

# Flip statistics for selected subgraphs
with torch.no_grad():
    orig_preds = []
    for data in dataset:
        x_in = data.x.to(DEVICE)
        logits = model(x_in, data.edge_index.to(DEVICE), batch_for(x_in))
        orig_preds.append(logits.cpu())
    orig_preds = torch.cat(orig_preds, dim=0).argmax(dim=1).numpy()

adv_sel_preds = preds_all[selected]
num_flips = int((orig_preds[selected] != adv_sel_preds).sum())
print(f"\nSelected subgraphs: {len(selected)}. Flipped after perturbation: {num_flips} ({100.0*num_flips/len(selected):.2f}%).")

# ============================================================
# Step 2: Stability Under Input Noise (on perturbed subgraphs)
# ============================================================
def stability_for_subgraph(idx, sigma, num_samples):
    """Compute avg L2 change in logits for noisy perturbations around perturbed subgraph."""
    data = dataset[idx]
    base_x = perturbed_map[idx].to(DEVICE)
    edge_index = data.edge_index.to(DEVICE)
    batch = batch_for(base_x)

    with torch.no_grad():
        f_orig = model(base_x, edge_index, batch).squeeze()

    diffs = []
    for _ in range(num_samples):
        noise = sigma * torch.randn_like(base_x).to(DEVICE)
        f_noisy = model(base_x + noise, edge_index, batch).squeeze()
        diffs.append(torch.norm(f_noisy - f_orig).item())
    return float(np.mean(diffs))

print("\nComputing Stability Under Input Noise (this may take a while)...")
t0 = time.time()
per_sample_info = []  # (idx, label, stability, rel_err)
for i, idx in enumerate(selected):
    s_val = stability_for_subgraph(idx, NOISE_SIGMA, NUM_NOISE_SAMPLES)
    re_vals = [stability_for_subgraph(idx, NOISE_SIGMA, NUM_NOISE_SAMPLES) for _ in range(RELERR_RESAMPLES)]
    s_ref = float(np.mean(re_vals))
    rel_err = abs(s_val - s_ref) / (abs(s_ref) + 1e-12)
    per_sample_info.append((int(idx), int(labels_np[idx]), s_val, rel_err))
    if (i+1) % 10 == 0:
        print(f"  processed {i+1}/{len(selected)} subgraphs...")
t1 = time.time()
print(f"Done SUIN computation. Time elapsed: {t1-t0:.1f}s")

# ============================================================
# Step 3: Aggregate and report
# ============================================================
clean_stats = [(i, s, e) for (i, l, s, e) in per_sample_info if l == 0]
troj_stats  = [(i, s, e) for (i, l, s, e) in per_sample_info if l == 1]

def aggs(stats):
    if not stats:
        return (0.0, 0.0, 0.0, 0.0)
    Ss = np.array([s for (_, s, _) in stats])
    Es = np.array([e for (_, _, e) in stats])
    return (Ss.mean(), Ss.std(), Es.mean(), Es.std())

cS_mean, cS_std, cE_mean, cE_std = aggs(clean_stats)
tS_mean, tS_std, tE_mean, tE_std = aggs(troj_stats)

print("\n--- Stability Under Input Noise (on perturbed subgraphs) ---")
print(f" Clean:  avg_stability={cS_mean:.4f} ± {cS_std:.4f}, avg_relerr={cE_mean:.4e} ± {cE_std:.4e}")
print(f" Trojan: avg_stability={tS_mean:.4f} ± {tS_std:.4f}, avg_relerr={tE_mean:.4e} ± {tE_std:.4e}")

print("\nSample preview (first 6): (idx, label, stability, rel_err)")
for p in per_sample_info[:6]:
    print(p)



--- Stability Under Input Noise (PGD-first, then metric) ---
Selected subgraphs: clean=11, trojan=20

============= Robustness Evaluation (Full Test Set: perturbed selected + others unperturbed) =============
Accuracy: 96.08%
Precision: 0.9651, Recall: 0.9608, F1: 0.9622

Classification report:
              precision    recall  f1-score   support

       clean     0.7692    0.9091    0.8333        11
      trojan     0.9888    0.9670    0.9778        91

    accuracy                         0.9608       102
   macro avg     0.8790    0.9381    0.9056       102
weighted avg     0.9651    0.9608    0.9622       102

Confusion Matrix:
[[10  1]
 [ 3 88]]

Selected subgraphs: 31. Flipped after perturbation: 3 (9.68%).

Computing Stability Under Input Noise (this may take a while)...
  processed 10/31 subgraphs...
  processed 20/31 subgraphs...
  processed 30/31 subgraphs...
Done SUIN computation. Time elapsed: 17.2s

--- Stability Under Input Noise (on perturbed subgraphs) ---
 Clean:  av